In [1]:
import os
import datetime
import random
import os.path

import pandas as pd
import numpy as np

#生成するデータファイルの格納先のpathを指定してください
FILE_PATH = '/Users/hiroshiono/ono_project/text' 

NUM_SHOP = 10

NUM_GOODS = 20

In [2]:
pd.__version__

'0.22.0'

In [3]:

start = datetime.datetime(2013, 1, 1)
end = datetime.datetime(2016, 6, 30)

date_list = []
while start < end:
    date_list.append(start.strftime('%Y/%m/%d'))
    start +=  datetime.timedelta(days=1)
    
NUM_SHOP_goods = NUM_SHOP + NUM_GOODS


In [4]:
random_id = random.sample(range(NUM_SHOP_goods), NUM_SHOP_goods)

id_shop = random_id[:NUM_SHOP]
id_goods = random_id[NUM_SHOP:]

In [5]:
shop_list_data_day = []
goods_list_data_day = []

for shop in id_shop:
    shopid_data = [shop]*len(id_goods)
    shop_list_data_day += shopid_data
    goods_list_data_day += id_goods

#
length_shop_goods_par_day = len(goods_list_data_day)


day_list_test_data = []
shop_list_test_data = []
goods_list_test_data = []


for day in date_list:
    day_test_data = [day]*length_shop_goods_par_day
    day_list_test_data += day_test_data
    shop_list_test_data += shop_list_data_day
    goods_list_test_data += goods_list_data_day
    
    
    

In [6]:
main_df = pd.DataFrame({
                      'shop_id' : shop_list_test_data,
                      'goods_id':goods_list_test_data,
                      'sale':np.random.rand(len(day_list_test_data)) * 1000,
                      'sales_quantity':np.random.rand(len(day_list_test_data)) * 1000
                     }, index=pd.to_datetime(day_list_test_data))
#保存
main_df.to_csv( FILE_PATH)

In [7]:

getsize_df=os.path.getsize(FILE_PATH )*9.536743164e-7


def check_df_size(_getsize_df):
    if _getsize_df>10:
        print("作成されたcsvは {:.2f}MBなので正常終了".format(_getsize_df))
        return True
    else:
        print("作成されたcsvは {:.2f}MBなので、あと {:.2f}MB足りません".format(_getsize_df,500-_getsize_df))
        raise Exception('capacity lack')
        
check_df_size(getsize_df)

作成されたcsvは 12.83MBなので正常終了


True

In [8]:
df_sum_month_group = main_df.groupby(pd.Grouper(freq='MS'))

df_month_shop_list =[]
df_month_goods_list =[]

for month in df_sum_month_group.sum().index[:]:
    df_month = df_sum_month_group.get_group(month)
    
    df_month_shop_group = df_month.groupby('shop_id').sum()
    df_month_goods_group = df_month.groupby('goods_id').sum()
    
    df_month_shop_group['date'] = month
    df_month_goods_group['date'] = month
    
    df_month_shop_list.append(df_month_shop_group.loc[:, ['date','sale']])
    df_month_goods_list.append(df_month_goods_group.loc[:, ['date','sale']])
    
df_month_shop = pd.concat(df_month_shop_list).reset_index().loc[:,['date','shop_id','sale']]
df_month_goods = pd.concat(df_month_goods_list).reset_index().loc[:,['date','goods_id','sale']]


In [9]:

pd.options.display.precision = 10
def get_sale_division(_x):
    sale,sales_quantity = _x
    
    return float(format(sale/sales_quantity, '.10f'))


df_day_group = main_df.groupby(pd.Grouper(freq='D'))

df_goods_mean_list = []
for day in df_day_group.sum().index[:]:
    
    df_day_division_list = []
    df_day = df_day_group.get_group(day)
    df_day_shop_group = df_day.groupby('shop_id')
    for shop_id in id_shop[:]:
        df_day_shop = df_day_shop_group.get_group(shop_id).copy()
        df_day_shop['sale/sale_quantity'] = df_day_shop.loc[:,['sale','sales_quantity']].apply(get_sale_division,axis=1)
        df_day_division_list.append(df_day_shop)
    df_shop_id_day_mean = pd.concat(df_day_division_list).groupby('goods_id').mean()
    df_shop_id_day_mean['day'] = day
    df_goods_mean_list.append(df_shop_id_day_mean)
    

df_goods_mean = pd.concat(df_goods_mean_list).reset_index().loc[:, ['day','goods_id','sale/sale_quantity']]


In [10]:
main_df.head()

,goods_id,sale,sales_quantity,shop_id
2013-01-01,0,734.0047041118,976.1966801310,18
2013-01-01,14,507.6459009329,325.1486566567,18
2013-01-01,25,236.9122156336,720.4011480627,18
2013-01-01,29,162.0440895326,496.9328550451,18
2013-01-01,23,659.9186226980,235.2966616013,18


In [11]:
df_month_shop.head()

,date,shop_id,sale
0,2013-01-01,7,310289.2779826038
1,2013-01-01,8,305936.1418522086
2,2013-01-01,10,298205.4896665026
3,2013-01-01,12,317410.8229606421
4,2013-01-01,15,311003.0594466545


In [12]:
df_month_goods.head()

,date,goods_id,sale
0,2013-01-01,0,152992.1186705921
1,2013-01-01,1,160529.3404542719
2,2013-01-01,2,158531.4720524671
3,2013-01-01,3,154613.8969377864
4,2013-01-01,4,155174.0075894637


In [13]:
df_goods_mean.head()

,day,goods_id,sale/sale_quantity
0,2013-01-01,0,1.5124484920
1,2013-01-01,1,15.9502037919
2,2013-01-01,2,1.1886903920
3,2013-01-01,3,11.3607699446
4,2013-01-01,4,1.6588192187
